<a href="https://colab.research.google.com/github/injoon-pij/pytorch-learning/blob/master/pytorch_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) RNN (character level)

## (1)

문자 시퀀스 apple을 입력받으면 pple!를 출력하는 RNN을 구현

### 1.1 Data preprocessing

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [ ]:
# 입력 데이터와 레이블 데이터에 대한 문자 집합(voabulary)
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str+label_str)))
vocab_size = len(char_vocab)

print ('문자 집합 : {}'.format(char_vocab))
print ('문자 집합의 크기 : {}'.format(vocab_size))

문자 집합 : ['!', 'a', 'e', 'l', 'p']
문자 집합의 크기 : 5


In [ ]:
input_size = vocab_size # 입력의 크기는 문자 집합의 크기 (원-핫 벡터 사용)
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [ ]:
# 문자에 고유한 정수 인덱스 부여
char_to_index = dict((c, i) for i, c in enumerate(char_vocab))
print(char_to_index)

# 인덱스-문자 dict
index_to_char = {}
for key, value in char_to_index.items():
    index_to_char[value] = key
print(index_to_char)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}
{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [ ]:
# 입력 데이터와 레이블 데이터 정수로 맵핑
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


In [ ]:
# nn.RNN()은 기본적으로 3차원 텐서를 입력받기 때문에 배치 차원을 추가
# 텐서 연산인 unsqueeze(0)를 통해 해결할 수도
x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


In [ ]:
# 입력 데이터를 원핫 벡터로 변환
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [ ]:
# 입력 데이터와 레이블 데이터를 텐서로 변환
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])


### 1.2 Modeling

In [ ]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True) # RNN 셀 구현
        self.fc = nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현

    def forward(self, x): # 구현한 RNN 셀과 출력층을 연결
        x, _status = self.rnn(x)
        x = self.fc(x) # x.shape = (batch_size, sequence_length, hidden_size)
        return x

In [ ]:
net = Net(input_size, hidden_size, output_size)

outputs = net(X)
print(outputs.shape) # 3차원 텐서 출력 (batch_size, sequence_length, output_size)

torch.Size([1, 5, 5])


In [ ]:
import numpy as np

In [ ]:
# loss function
criterion = nn.CrossEntropyLoss()

# optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

# train
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, output_size), Y.view(-1)) # view : Batch 차원 제거
    loss.backward()
    optimizer.step() 

    # 최종 예측값인 각 time-step 별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택
    result = outputs.data.numpy().argmax(axis=2) # outputs.data.numpy() : tensor를 numpy로 변환
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  1.5707448720932007 prediction:  [[4 2 2 2 3]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  peeel
1 loss:  1.2935644388198853 prediction:  [[4 4 2 2 2]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppeee
2 loss:  1.0445367097854614 prediction:  [[4 4 3 2 2]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pplee
3 loss:  0.8069427609443665 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
4 loss:  0.6217581629753113 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
5 loss:  0.4866600036621094 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
6 loss:  0.3835570812225342 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
7 loss:  0.29876938462257385 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
8 loss:  0.22924959659576416 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
9 loss:  0.17240861058235168 prediction:  [[4 4 3 2 0

## (2)

더 많은 데이터에 대해 문자 단위 RNN을 구현

### 2.1 Data preprocessing

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [ ]:
char_set = list(set(sentence)) # vocabulary
char_dic = {c: i for i, c in enumerate(char_set)} # 각 문자에 정수 인코딩
dic_size = len(char_dic)
print('문자 집합의 크기 : {}'.format(dic_size))
print(char_dic)

문자 집합의 크기 : 25
{'b': 0, 'g': 1, 'p': 2, 'l': 3, 'i': 4, 't': 5, 'u': 6, 'h': 7, 'o': 8, '.': 9, 'k': 10, 'w': 11, 'm': 12, 's': 13, 'd': 14, 'y': 15, 'c': 16, ' ': 17, 'n': 18, 'a': 19, ',': 20, 'f': 21, 'e': 22, 'r': 23, "'": 24}


In [ ]:
# 하이퍼파라미터 설정
hidden_size = dic_size
sequence_length = 10  # 임의 숫자 지정
learning_rate = 0.1

In [ ]:
# 데이터 구성
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i : i+sequence_length]
    y_str = sentence[i+1: i+sequence_length+1]
    print(i, x_str, '->', y_str)

    x_data.append([char_dic[c] for c in x_str])  # x str to index
    y_data.append([char_dic[c] for c in y_str])  # y str to index

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [ ]:
print(x_data[0]) # if you wan
print(y_data[0]) # f you want

[4, 21, 17, 15, 8, 6, 17, 11, 19, 18]
[21, 17, 15, 8, 6, 17, 11, 19, 18, 5]


In [ ]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data] # x 데이터는 원-핫 인코딩
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([170, 10, 25])
레이블의 크기 : torch.Size([170, 10])


### 2.2 Modeling

In [ ]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, layers): # 현재 hidden_size는 dic_size와 같음.
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [ ]:
net = Net(dic_size, hidden_size, 2) # 이번에는 층을 두 개 쌓습니다.
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [ ]:
outputs = net(X)
print(outputs.shape) # 3차원 텐서

torch.Size([170, 10, 25])


In [ ]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X) # input_data size (170, 10, 25) 
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    # results의 텐서 크기는 (170, 10)
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오지만
            predict_str += ''.join([char_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += char_set[result[-1]]

    if i % 10 == 0:
      print("epoch {}".format(i), predict_str)

epoch 0 ggeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeaeeeeeeeeeeeeageeeeeeeeegeeeeeeeeeegeeeeeeeeeeegeeeeeeeeeeeegeeeeeeeeeeeeeeeeeeegeeeeeeeeeeegeeeeeeeeeeeegeeeeeeeeeeeeeeegeeeeeeeeeegeeeeegee
epoch 10   dp i a d d  t d e s i tas d e t t dt d  t    t t e   e  d d s   it w    t s w e t t et t w e  w ect t d d n s d d d sa m d d   a  m w e'  t a nps d  aate   t      t em  t ea a  
epoch 20 gyteo tand to puile tod ips ton't doum ap ponp e theothem thrpo ees, wo p tnd ton't dssigo them to cs tnd torkt tut duthem toncs them tonpon' poe themsnd ess tu ens ty am themsens
epoch 30 gyy uhwan' to build aothip, don't urum up people th ether th collect word wnd won't ussign them tosks and work, dut rather thach them to bong for themsnd e s tmmensity on themseas
epoch 40 thy u want to build a ship, don't urum up people together to collect wood and won't ussign them tosks and work, dut rather toach them to long for themendless immensity of themeea.
epoch 50 t you want to build a ship, don't drum up peopl

# 2) RNN (word level)

__'Repeat is the best medicine for'__을 입력받으면 __'is the best medicine for memory'__를 출력하는 RNN 모델 구현

## 2.1 Data preprocessing

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

In [1]:
sentence = "Repeat is the best medicine for memory".split()

# vocab
vocab = list(set(sentence))
print(vocab)

# 단어에 고유한 정수 부여
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}
word2index['<unk>']=0
print(word2index)

# idx2word dictionary 
index2word = {v: k for k, v in word2index.items()}
print(index2word)

['for', 'memory', 'the', 'medicine', 'best', 'Repeat', 'is']
{'for': 1, 'memory': 2, 'the': 3, 'medicine': 4, 'best': 5, 'Repeat': 6, 'is': 7, '<unk>': 0}
{1: 'for', 2: 'memory', 3: 'the', 4: 'medicine', 5: 'best', 6: 'Repeat', 7: 'is', 0: '<unk>'}


In [4]:
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환. 
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq

In [5]:
X, Y = build_data(sentence, word2index)
print(X)
print(Y)

tensor([[6, 7, 3, 5, 4, 1]])
tensor([[7, 3, 5, 4, 1, 2]])


## 2.2 Modeling

이전 모델들과 달라진 점은 임베딩 층을 추가했다는 것. (nn.Embedding())

임베딩층은 크게 두 가지 인자를 받는데 첫번째 인자는 단어장의 크기이며, 두번째 인자는 임베딩 벡터의 차원임

In [6]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings = vocab_size, embedding_dim = input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, batch_first = batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) # 출력 크기는 단어 집합의 크기와 같아야함

    def forward(self, x):
        # (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # output : (배치 크기, 시퀀스 길이, 은닉층 크기), hidden : (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        # (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

In [7]:
# hyperparameter
vocab_size = len(word2index)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. <unk> 토큰을 크기에 포함한다.
input_size = 5  # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20  # RNN의 은닉층 크기

# model
model = Net(vocab_size, input_size, hidden_size, batch_first=True)

# cost function
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 레이블값은 원-핫 인코딩 안 해도 됨.

# optimizer
optimizer = optim.Adam(params=model.parameters())

In [8]:
output = model(X)
print(output)
print(output.shape)

tensor([[ 0.1243,  0.0961, -0.0696, -0.1287,  0.3780,  0.0083,  0.0472,  0.0020],
        [ 0.1929,  0.2069, -0.3637, -0.2539,  0.1969,  0.3460,  0.0099, -0.2718],
        [ 0.2203,  0.3917, -0.3413, -0.1028,  0.1934,  0.0407, -0.2451,  0.0736],
        [ 0.2316,  0.1667,  0.0456, -0.1516,  0.2706,  0.1618,  0.1669, -0.1120],
        [ 0.1085,  0.3258, -0.3764, -0.0701,  0.1445,  0.2894, -0.5274,  0.1147],
        [ 0.3694,  0.0239, -0.3853,  0.0057,  0.1107,  0.5964, -0.3665, -0.0027]],
       grad_fn=<ViewBackward>)
torch.Size([6, 8])


In [9]:
# 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y: [index2word.get(x) for x in y]

In [10]:
# 훈련 시작
for step in range(201):
    optimizer.zero_grad()
    output = model(X)

    loss = loss_function(output, Y.view(-1))
    loss.backward()
    optimizer.step()
    
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201] 2.1465 
Repeat medicine best for medicine for best

[41/201] 1.5476 
Repeat medicine for best medicine for memory

[81/201] 0.9017 
Repeat is the best medicine for memory

[121/201] 0.4554 
Repeat is the best medicine for memory

[161/201] 0.2289 
Repeat is the best medicine for memory

[201/201] 0.1312 
Repeat is the best medicine for memory

